# Data Generation: Constitutional Preference Pairs

This notebook generates preference pairs for training Constitutional AI models.

## What it does:
1. Loads prompts from AILuminate benchmark
2. Generates baseline responses using a base model
3. Critiques responses using contemplative principles
4. Generates revised responses
5. Creates preference pairs (rejected vs chosen)
6. Creates train/test splits
7. Syncs results to S3

## Parameters to configure:
- Number of prompts
- Hazard categories to include
- Model to use for generation
- Constitution file


In [ ]:
import sys
import os
from pathlib import Path
import yaml

# Setup
os.chdir('..')
sys.path.insert(0, str(Path.cwd() / 'src'))

with open('configs/sagemaker_configs.yaml', 'r') as f:
    config = yaml.safe_load(f)

S3_BUCKET = config['s3']['bucket']
print(f"Working directory: {os.getcwd()}")
print(f"S3 Bucket: {S3_BUCKET}")


## Configuration

Adjust these parameters based on your needs:


In [ ]:
# Generation Configuration
CONFIG = {
    # Data source
    'max_prompts': 100,  # Number of prompts to process (100 prompts = 400 pairs with 4 principles)
    'hazard_categories': ['vcr', 'cse', 'hte', 'ssh'],  # AILuminate hazard types
    'persona_types': None,  # None = all persona types
    
    # Model settings
    'model': 'qwen2_7b',  # Options: qwen2_0_5b, qwen2_1_5b, qwen2_7b
    'device': 'cuda',
    'max_memory_gb': 20.0,
    
    # Constitution
    'constitution': 'data/constitutions/contemplative_principles.md',
    'principles': None,  # None = use all principles
    
    # Output
    'output_file': 'results/preference_pairs_100.jsonl',
    'split_config': 'data/splits/default_split.json',
    'test_size': 0.1,  # 10% for test set
}

# Print configuration
print("Data Generation Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")


## Generate Preference Pairs

This will take some time depending on the number of prompts and model size.


In [ ]:
# Build command
cmd_parts = [
    "python scripts/generate_cai_data.py",
    "--use-ailuminate",
    f"--constitution {CONFIG['constitution']}",
    f"--model {CONFIG['model']}",
    f"--max-prompts {CONFIG['max_prompts']}",
    f"--device {CONFIG['device']}",
    f"--max-memory-gb {CONFIG['max_memory_gb']}",
    f"--output {CONFIG['output_file']}",
    "--create-split",
    f"--test-size {CONFIG['test_size']}",
    f"--split-config {CONFIG['split_config']}",
]

if CONFIG['hazard_categories']:
    cmd_parts.append(f"--hazard-categories {' '.join(CONFIG['hazard_categories'])}")

if CONFIG['persona_types']:
    cmd_parts.append(f"--persona-types {' '.join(CONFIG['persona_types'])}")

if CONFIG['principles']:
    cmd_parts.append(f"--principles {' '.join(CONFIG['principles'])}")

command = " \\\n    ".join(cmd_parts)
print("Running command:")
print(command)
print("\n" + "="*50)


In [ ]:
# Execute generation
!{command}


## Analyze Generated Data


In [ ]:
import json
import pandas as pd
from collections import Counter

# Load generated pairs
pairs = []
output_path = Path(CONFIG['output_file'])
if output_path.exists():
    with open(output_path, 'r') as f:
        for line in f:
            pairs.append(json.loads(line))

print(f"Generated {len(pairs)} preference pairs")

# Analyze by principle
principles = [p['principle'] for p in pairs]
principle_counts = Counter(principles)
print(f"\nBreakdown by principle:")
for principle, count in principle_counts.items():
    print(f"  {principle}: {count}")

# Show examples
print(f"\n{'='*60}")
print("Example Preference Pair:")
print(f"{'='*60}")
if pairs:
    example = pairs[0]
    print(f"\nPrinciple: {example['principle']}")
    print(f"\nPrompt:\n{example['prompt'][:300]}...")
    print(f"\nRejected Response:\n{example['rejected'][:300]}...")
    print(f"\nChosen Response:\n{example['chosen'][:300]}...")


## View Train/Test Split


In [ ]:
# Load split configuration
split_path = Path(CONFIG['split_config'])
if split_path.exists():
    with open(split_path, 'r') as f:
        split_info = json.load(f)
    
    print("Train/Test Split Information:")
    print(f"  Total prompts: {split_info['n_prompts']}")
    print(f"  Train prompts: {len(split_info['train_prompt_ids'])}")
    print(f"  Test prompts: {len(split_info['test_prompt_ids'])}")
    print(f"  Test size: {split_info['test_size']}")
    print(f"  Random seed: {split_info['random_state']}")
    print(f"\nWith {len(principle_counts)} principles per prompt:")
    print(f"  Expected train pairs: {len(split_info['train_prompt_ids']) * len(principle_counts)}")
    print(f"  Expected test pairs: {len(split_info['test_prompt_ids']) * len(principle_counts)}")
else:
    print(f"Split configuration not found at {split_path}")


## Sync to S3

Save the generated data to S3 for persistence and sharing.


In [ ]:
from utils.sagemaker_utils import sync_to_s3

if S3_BUCKET != "your-bucket-contemplative-ai":
    print("Syncing results to S3...")
    
    # Sync preference pairs
    output_filename = Path(CONFIG['output_file']).name
    s3_data_path = f"s3://{S3_BUCKET}/results/preference_pairs/{output_filename}"
    success = sync_to_s3(CONFIG['output_file'], s3_data_path)
    
    # Sync split configuration
    split_filename = Path(CONFIG['split_config']).name
    s3_split_path = f"s3://{S3_BUCKET}/data/splits/{split_filename}"
    success = sync_to_s3(CONFIG['split_config'], s3_split_path) and success
    
    if success:
        print(f"\n✅ Results synced to S3:")
        print(f"   Data: {s3_data_path}")
        print(f"   Split: {s3_split_path}")
    else:
        print("\n⚠️ Some files failed to sync")
else:
    print("⚠️ Skipping S3 sync - configure bucket in configs/sagemaker_configs.yaml")


## Summary

✅ Data generation complete!

### Generated Files:
- **Preference pairs**: `{CONFIG['output_file']}`
- **Split config**: `{CONFIG['split_config']}`

### Next Steps:
1. Review the generated pairs above
2. Use `02_training.ipynb` to train a model with this data
3. Or generate more data with different settings by rerunning this notebook
